In [1]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 200)

In [3]:
def percent_of_nan_rows(dataset:pd.DataFrame, column:str) -> float:
    return len(X[X[column].isna()]) / len(X) * 100

In [4]:
myocardial_infarction_complications = fetch_ucirepo(id=579)
X = myocardial_infarction_complications.data.features
y = myocardial_infarction_complications.data.targets

In [5]:
X = X.fillna(np.nan)

In [6]:
vars_ = pd.DataFrame(myocardial_infarction_complications.variables)

In [7]:
vars_

,name,role,type,demographic,description,units,missing_values
0,ID,ID,Integer,None,Record ID (ID): Unique identifier. Cannot be r...,None,no
1,AGE,Feature,Integer,Age,Age of patient.,None,no
2,SEX,Feature,Binary,Sex,"0: female, 1: male",None,no
3,INF_ANAM,Feature,Categorical,None,Quantity of myocardial infarctions in the anam...,None,yes
4,STENOK_AN,Feature,Categorical,None,Exertional angina pectoris in the anamnesis. \...,None,yes
...,...,...,...,...,...,...,...
119,DRESSLER,Target,Binary,None,Dressler syndrome,None,no
120,ZSN,Target,Binary,None,Chronic heart failure,None,no
121,REC_IM,Target,Binary,None,Relapse of the myocardial infarction,None,no
122,P_IM_STEN,Target,Binary,None,Post-infarction angina,None,no


In [8]:
cat_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Categorical')]['name'].tolist()
bin_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Binary')]['name'].tolist()
int_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Integer')]['name'].tolist()
float_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Continuous')]['name'].tolist()

In [9]:
X.drop(index = X[X.count(axis=1)<len(X.columns)*0.7].index, inplace=True)

In [10]:
for column in X.columns:
    if percent_of_nan_rows(X, column) > 75:
        X.drop(columns = [column], inplace=True)
        try:
            bin_cols.remove(column)
        except:
            pass
        try:
            cat_cols.remove(column)
        except:
            pass
        try:
            int_cols.remove(column)
        except:
            pass
        try:
            float_cols.remove(column)
        except:
            pass
    else:
        if column in cat_cols or column in bin_cols:
            names = X[column].value_counts().index.tolist()
            weights = X[column].value_counts().tolist()/X[column].count()
            X[column] = X[column].apply(lambda l: l if not np.isnan(l) else np.random.choice(names, p=weights))
            pass
        else:
            X[column] = X[column].apply(lambda l: l if not np.isnan(l) else X[column].mean() + np.random.random()*2*X[column].std())
            X[column] = X[column].apply(lambda l: (l-X[column].mean())/X[column].std())
            pass

In [11]:
X[bin_cols] = X[bin_cols].applymap(lambda x: bool(x))

In [12]:
X = pd.get_dummies(data = X, columns = cat_cols)

In [13]:
bin_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Binary')]['name'].tolist()
cat_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Categorical')]['name'].tolist()

In [14]:
y[bin_cols_preds] = y[bin_cols_preds].applymap(lambda x: bool(x))

/tmp/ipykernel_80219/2638328380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[bin_cols_preds] = y[bin_cols_preds].applymap(lambda x: bool(x))


In [15]:
#y = pd.get_dummies(data = y, columns = cat_cols_preds)

In [16]:
data = X.merge(y, left_index=True, right_index=True)

In [17]:
X = data[X.columns]

In [18]:
y = data[y.columns]

In [19]:
X

,AGE,SEX,SIM_GIPERT,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_KBRIG,D_AD_KBRIG,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,MP_TP_POST,SVT_POST,GT_POST,FIB_G_POST,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,NA_KB,NOT_NA_KB,LID_KB,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,INF_ANAM_0.0,INF_ANAM_1.0,INF_ANAM_2.0,INF_ANAM_3.0,STENOK_AN_0.0,STENOK_AN_1.0,STENOK_AN_2.0,STENOK_AN_3.0,STENOK_AN_4.0,STENOK_AN_5.0,STENOK_AN_6.0,FK_STENOK_0.0,FK_STENOK_1.0,FK_STENOK_2.0,FK_STENOK_3.0,FK_STENOK_4.0,IBS_POST_0.0,IBS_POST_1.0,IBS_POST_2.0,GB_0.0,GB_1.0,GB_2.0,GB_3.0,DLIT_AG_0.0,DLIT_AG_1.0,DLIT_AG_2.0,DLIT_AG_3.0,DLIT_AG_4.0,DLIT_AG_5.0,DLIT_AG_6.0,DLIT_AG_7.0,ZSN_A_0.0,ZSN_A_1.0,ZSN_A_2.0,ZSN_A_3.0,ZSN_A_4.0,ant_im_0.0,ant_im_1.0,ant_im_2.0,ant_im_3.0,ant_im_4.0,lat_im_0.0,lat_im_1.0,lat_im_2.0,lat_im_3.0,lat_im_4.0,inf_im_0.0,inf_im_1.0,inf_im_2.0,inf_im_3.0,inf_im_4.0,post_im_0.0,post_im_1.0,post_im_2.0,post_im_3.0,post_im_4.0,TIME_B_S_1.0,TIME_B_S_2.0,TIME_B_S_3.0,TIME_B_S_4.0,TIME_B_S_5.0,TIME_B_S_6.0,TIME_B_S_7.0,TIME_B_S_8.0,TIME_B_S_9.0,R_AB_1_n_0.0,R_AB_1_n_1.0,R_AB_1_n_2.0,R_AB_1_n_3.0,R_AB_2_n_0.0,R_AB_2_n_1.0,R_AB_2_n_2.0,R_AB_2_n_3.0,R_AB_3_n_0.0,R_AB_3_n_1.0,R_AB_3_n_2.0,R_AB_3_n_3.0,NOT_NA_1_n_0.0,NOT_NA_1_n_1.0,NOT_NA_1_n_2.0,NOT_NA_1_n_3.0,NOT_NA_1_n_4.0
0,1.339785,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,1.081391,-0.674946,1.306107,0.800295,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0.474344,False,0.023567,1.736922,1.271809,-0.297646,0.116294,True,False,False,False,-0.623514,-0.319814,-0.279642,-0.340588,-0.308302,True,False,False,True,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False
1,-0.603448,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,-0.060225,0.389234,-0.646713,0.235334,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,-1.108184,False,-0.884724,-0.426280,-0.572157,-0.356108,-1.018532,True,False,True,False,-0.623514,-0.319814,-0.279642,-0.340588,-0.308302,True,False,True,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,Fals

In [20]:
y

,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,False,False,False,False,False,False,False,False,False,False,False,0
1,False,False,False,False,False,False,False,False,False,False,False,0
2,False,False,False,False,False,False,False,False,False,False,False,0
3,False,False,False,False,False,False,False,False,True,False,False,0
4,False,False,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1694,False,False,False,False,False,False,True,False,False,False,False,3
1695,False,False,False,False,True,False,True,False,False,False,False,3
1696,False,False,False,False,False,False,False,False,False,False,False,1
1698,False,False,False,False,False,True,False,False,False,False,False,1


In [27]:
data = pd.concat([data[data['LET_IS'] != 0], data[data['LET_IS'] == 0].sample(int(len(data[data['LET_IS'] != 0])*1.5))])

In [28]:
data

,AGE,SEX,SIM_GIPERT,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_KBRIG,D_AD_KBRIG,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,MP_TP_POST,SVT_POST,GT_POST,FIB_G_POST,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,NA_KB,NOT_NA_KB,LID_KB,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,INF_ANAM_0.0,INF_ANAM_1.0,INF_ANAM_2.0,INF_ANAM_3.0,STENOK_AN_0.0,STENOK_AN_1.0,STENOK_AN_2.0,STENOK_AN_3.0,STENOK_AN_4.0,STENOK_AN_5.0,STENOK_AN_6.0,FK_STENOK_0.0,FK_STENOK_1.0,FK_STENOK_2.0,FK_STENOK_3.0,FK_STENOK_4.0,IBS_POST_0.0,IBS_POST_1.0,IBS_POST_2.0,GB_0.0,GB_1.0,GB_2.0,GB_3.0,DLIT_AG_0.0,DLIT_AG_1.0,DLIT_AG_2.0,DLIT_AG_3.0,DLIT_AG_4.0,DLIT_AG_5.0,DLIT_AG_6.0,DLIT_AG_7.0,ZSN_A_0.0,ZSN_A_1.0,ZSN_A_2.0,ZSN_A_3.0,ZSN_A_4.0,ant_im_0.0,ant_im_1.0,ant_im_2.0,ant_im_3.0,ant_im_4.0,lat_im_0.0,lat_im_1.0,lat_im_2.0,lat_im_3.0,lat_im_4.0,inf_im_0.0,inf_im_1.0,inf_im_2.0,inf_im_3.0,inf_im_4.0,post_im_0.0,post_im_1.0,post_im_2.0,post_im_3.0,post_im_4.0,TIME_B_S_1.0,TIME_B_S_2.0,TIME_B_S_3.0,TIME_B_S_4.0,TIME_B_S_5.0,TIME_B_S_6.0,TIME_B_S_7.0,TIME_B_S_8.0,TIME_B_S_9.0,R_AB_1_n_0.0,R_AB_1_n_1.0,R_AB_1_n_2.0,R_AB_1_n_3.0,R_AB_2_n_0.0,R_AB_2_n_1.0,R_AB_2_n_2.0,R_AB_2_n_3.0,R_AB_3_n_0.0,R_AB_3_n_1.0,R_AB_3_n_2.0,R_AB_3_n_3.0,NOT_NA_1_n_0.0,NOT_NA_1_n_1.0,NOT_NA_1_n_2.0,NOT_NA_1_n_3.0,NOT_NA_1_n_4.0,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
1426,0.279840,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.103064,-0.158631,2.607987,0.800295,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.537723,False,0.846224,-0.124979,0.708611,0.129696,0.367488,True,True,False,False,0.663260,1.579417,1.363178,0.926254,1.489335,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2
1427,-0.780105,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0.660966,1.157314,0.004227,0.235334,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,-0.448797,False,-0.733342,-1.004456,-0.717597,0.783898,-1.018532,True,True,False,False,1.950035,-0.319814,-0.279642,-0.340588,-0.308302,True,False,True,True,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False